In [1]:
library("survival")
library("arrow")
library("tidyverse")
library("feather")
library("survminer")
library("ggplot2")
library("tableone")
library("ggfortify")


Attaching package: ‘arrow’


The following object is masked from ‘package:utils’:

    timestamp


── Attaching packages ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘feather’


The following objects are masked from ‘package:arrow’:

    read_feather, write_feather


Loading required package: ggpubr


Attaching package: ‘survminer’


The following object is masked from ‘package:survival’:

    myeloma




In [2]:
tx_df <-read_parquet("/home/project/data/students/srtr/tx_cohort.parquet")
head(tx_df)

CAN_ABO,CAN_ANGINA,REC_DGN,CAN_DIAB_TY,CAN_GENDER,CAN_HGT_CM,CAN_MALIG,CAN_RACE,CAN_WGT_KG,DONOR_ID,⋯,REC_PREV_GRAFT1_DT,REC_GRAFT_STAT,REC_HGT_WGT_DT,REC_PROD_URINE_GT40_24HRS,REC_TX_DT,TFL_DEATH_DT,TFL_GRAFT_DT,TFL_LAFUDATE,TX_ID,TFL_COD
<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<int>,⋯,<dttm>,<chr>,<dttm>,<chr>,<dttm>,<dttm>,<dttm>,<dttm>,<int>,<dbl>
B,NA,3070,3,M,170.20,N,16,63.500,421873,⋯,NA,Y,NA,Y,2012-12-20 01:00:00,NA,2018-12-27 01:00:00,2018-12-27 01:00:00,1590873,NA
A,NA,3034,1,M,180.34,N,8,88.900,421665,⋯,NA,Y,NA,Y,2013-01-22 01:00:00,NA,NA,2021-01-23 01:00:00,1593148,NA
O,NA,3040,1,M,180.00,N,8,86.680,420796,⋯,NA,Y,NA,Y,2012-11-26 01:00:00,NA,2017-10-12 02:00:00,2017-11-16 01:00:00,1588938,NA
O,NA,3070,3,M,172.00,N,2000,86.000,421449,⋯,NA,Y,NA,NA,2012-12-07 01:00:00,2021-06-17 02:00:00,NA,2021-06-17 02:00:00,1589878,3500
B,NA,3001,1,F,157.48,N,16,56.699,421690,⋯,NA,Y,NA,Y,2013-02-13 01:00:00,NA,NA,2021-02-04 01:00:00,1594745,NA
A,NA,3006,1,M,177.80,N,8,71.670,422794,⋯,NA,Y,NA,NA,2013-01-01 01:00:00,NA,NA,2020-10-24 02:00:00,1591556,NA


In [3]:
colnames(tx_df)

[1] "CAN_ABO"                        "CAN_ANGINA"                    
 [3] "REC_DGN"                        "CAN_DIAB_TY"                   
 [5] "CAN_GENDER"                     "CAN_HGT_CM"                    
 [7] "CAN_MALIG"                      "CAN_RACE"                      
 [9] "CAN_WGT_KG"                     "DONOR_ID"                      
[11] "DON_ABO"                        "DON_AGE"                       
[13] "DON_AGE_IN_MONTHS"              "DON_GENDER"                    
[15] "REC_ACUTE_REJ_BIOPSY_CONFIRMED" "REC_ACUTE_REJ_EPISODE"         
[17] "REC_AGE_AT_TX"                  "REC_AGE_IN_MONTHS_AT_TX"       
[19] "REC_A_MM_EQUIV_CUR"             "REC_A_MM_EQUIV_TX"             
[21] "REC_BMI"                        "REC_B_MM_EQUIV_CUR"            
[23] "REC_B_MM_EQUIV_TX"              "REC_DR_MM_EQUIV_CUR"           
[25] "REC_DR_MM_EQUIV_TX"             "REC_MM_EQUIV_CUR"              
[27] "REC_MM_EQUIV_TX"                "REC_COD"                       
[29] "REC_COD2"                       "REC_COD3"                      
[31] "REC_COLD_ISCH_TM"               "REC_CREAT"                     
[33] "REC_CREAT_DECLINE_GE25"         "REC_DISCHRG_CREAT"             
[35] "REC_DIAL_DT"                    "REC_DISCHRG_DT"                
[37] "REC_FAIL_DT"                    "REC_FAIL_CAUSE_TY"             
[39] "REC_PREV_GRAFT1_DT"             "REC_GRAFT_STAT"                
[41] "REC_HGT_WGT_DT"                 "REC_PROD_URINE_GT40_24HRS"     
[43] "REC_TX_DT"                      "TFL_DEATH_DT"                  
[45] "TFL_GRAFT_DT"                   "TFL_LAFUDATE"                  
[47] "TX_ID"                          "TFL_COD"

In [4]:
# take columnnames suggested by nephrologist and clinical reasoning and removing multiple mismatch parameter to only get the overall mismatch percentage because of obious multicolinearity
tx_df <- tx_df %>% select(-c('CAN_HGT_CM','CAN_WGT_KG','DONOR_ID','DON_AGE_IN_MONTHS','REC_AGE_IN_MONTHS_AT_TX','REC_AGE_IN_MONTHS_AT_TX','REC_FAIL_DT','REC_FAIL_CAUSE_TY','REC_PREV_GRAFT1_DT','REC_GRAFT_STAT','REC_HGT_WGT_DT',
                             'TX_ID','TFL_COD','REC_COD','REC_COD2','REC_COD3','TFL_COD', 'CAN_RACE','REC_A_MM_EQUIV_CUR','REC_A_MM_EQUIV_TX','REC_BMI','REC_B_MM_EQUIV_CUR','REC_B_MM_EQUIV_TX','REC_DR_MM_EQUIV_CUR',
                             'REC_DR_MM_EQUIV_TX'))

In [5]:
colnames(tx_df)

[1] "CAN_ABO"                        "CAN_ANGINA"                    
 [3] "REC_DGN"                        "CAN_DIAB_TY"                   
 [5] "CAN_GENDER"                     "CAN_MALIG"                     
 [7] "DON_ABO"                        "DON_AGE"                       
 [9] "DON_GENDER"                     "REC_ACUTE_REJ_BIOPSY_CONFIRMED"
[11] "REC_ACUTE_REJ_EPISODE"          "REC_AGE_AT_TX"                 
[13] "REC_MM_EQUIV_CUR"               "REC_MM_EQUIV_TX"               
[15] "REC_COLD_ISCH_TM"               "REC_CREAT"                     
[17] "REC_CREAT_DECLINE_GE25"         "REC_DISCHRG_CREAT"             
[19] "REC_DIAL_DT"                    "REC_DISCHRG_DT"                
[21] "REC_PROD_URINE_GT40_24HRS"      "REC_TX_DT"                     
[23] "TFL_DEATH_DT"                   "TFL_GRAFT_DT"                  
[25] "TFL_LAFUDATE"

In [6]:
data <- tx_df%>%mutate(time = case_when(
  !is.na(TFL_DEATH_DT) & is.na(TFL_GRAFT_DT)~ difftime( TFL_DEATH_DT,REC_TX_DT, units = "days"),
    is.na(TFL_DEATH_DT) & !is.na(TFL_GRAFT_DT)~ difftime(TFL_GRAFT_DT,REC_TX_DT,  units = "days"),
    is.na(TFL_DEATH_DT) & is.na(TFL_GRAFT_DT)~ difftime(TFL_LAFUDATE,REC_TX_DT,  units = "days"),
  ))

# time: Survival time in days
# status: censoring status 1=censored, 2=graft failure
data$status <- 1
data$status[!is.na(data$TFL_DEATH_DT)]<-1
data$status[!is.na(data$TFL_GRAFT_DT)]<-2

In [7]:
colSums(is.na(data))
# drop columns with too much missing data
data <- data %>% select(-c('CAN_ANGINA', 'REC_ACUTE_REJ_BIOPSY_CONFIRMED','REC_CREAT_DECLINE_GE25','REC_PROD_URINE_GT40_24HRS'))

CAN_ABO                     CAN_ANGINA 
                             0                         365430 
                       REC_DGN                    CAN_DIAB_TY 
                          4061                          72073 
                    CAN_GENDER                      CAN_MALIG 
                             0                          72439 
                       DON_ABO                        DON_AGE 
                             9                             21 
                    DON_GENDER REC_ACUTE_REJ_BIOPSY_CONFIRMED 
                             1                         506772 
         REC_ACUTE_REJ_EPISODE                  REC_AGE_AT_TX 
                        192470                              1 
              REC_MM_EQUIV_CUR                REC_MM_EQUIV_TX 
                          5697                          40261 
              REC_COLD_ISCH_TM                      REC_CREAT 
                         49004                         122028 
        REC_CREAT_DECLINE_GE25              REC_DISCHRG_CREAT 
                        286905                          12929 
                   REC_DIAL_DT                 REC_DISCHRG_DT 
                         91128                           5885 
     REC_PROD_URINE_GT40_24HRS                      REC_TX_DT 
                        286047                              0 
                  TFL_DEATH_DT                   TFL_GRAFT_DT 
                        395034                         390251 
                  TFL_LAFUDATE                           time 
                             0                          23978 
                        status 
                             0

In [8]:
data$yeartx <- format(data$REC_TX_DT, format="%Y")
min(data$yeartx)

[1] "1987"

In [9]:
data$decade <- '80'
data$decade[data$yeartx>1989]<-'90'
data$decade[data$yeartx>1999]<- '2000'
data$decade[data$yeartx>2009]<- '2010'
data$decade[data$yeartx>2019]<- '2020'
data$decade <- as.factor(data$decade)

data <- data %>% select(-c("REC_DIAL_DT", "REC_DISCHRG_DT", "REC_TX_DT","yeartx"))
unique(data$decade)

[1] 2010 2000 90   80   2020
Levels: 2000 2010 2020 80 90

In [ ]:
colSums(is.na(data))

In [ ]:
data <- data %>% select(-c('TFL_DEATH_DT','TFL_GRAFT_DT','TFL_LAFUDATE' ))

In [ ]:
head(data)

Pick timeframe

In [ ]:
plotting <- data
plotting$time <- as.integer(plotting$time)
plotting$status <- as.character(plotting$status)
plotting$status[plotting$status==1]<- "Censored"
plotting$status[plotting$status==2]<- "Failure"
plotting$status <- as.factor(plotting$status)


ggplot(plotting, aes(x=time, color=status)) +
    geom_histogram(fill="white")+
    theme_classic()

In [ ]:
# first 13 years after tx
data <- subset(data, data$time <4745)

Cleaning and refactoring

In [ ]:
str(data)

In [ ]:
cols <- c("CAN_ABO","REC_DGN","CAN_DIAB_TY", "CAN_GENDER", "CAN_MALIG", "DON_ABO", "DON_GENDER","REC_ACUTE_REJ_EPISODE", 
         "REC_MM_EQUIV_CUR","REC_MM_EQUIV_TX")
data[cols] <- lapply(data[cols], factor)

str(data)

In [ ]:
## Create a TableOne object
tab1 <- CreateTableOne(vars = c('CAN_ABO','REC_DGN','CAN_DIAB_TY','CAN_GENDER','CAN_MALIG','DON_ABO','DON_AGE','DON_GENDER','REC_AGE_AT_TX','REC_MM_EQUIV_CUR',
            'REC_MM_EQUIV_TX','REC_COLD_ISCH_TM','REC_CREAT','REC_DISCHRG_CREAT',
            'decade','status'), strata = "status", 
                       data = data, factorVars = c('CAN_ABO','REC_DGN','CAN_DIAB_TY','CAN_GENDER','CAN_MALIG','DON_ABO','DON_GENDER','REC_MM_EQUIV_TX','REC_MM_EQUIV_CUR','decade'))

tab3Mat <- print(tab1,showAllLevels = FALSE, quote = FALSE, noSpaces = TRUE, printToggle = FALSE)

write.csv(tab3Mat, 'myTable.csv')

In [ ]:
# univariate cox to select variables
# covariates
covariates <- colnames(data)
covariates <- covariates[!covariates %in% c("time","status")]

#making formulas
univ_formulas <- sapply(covariates,function(x)as.formula(paste('Surv(time,status)~',x))
)
#making a list of models
univ_models <- lapply(univ_formulas, function(x){coxph(x,data=data)})

#extract data (here I've gone for HR and confint)
univ_results <- lapply(univ_models,function(x){cbind(exp(cbind(coef(x),confint(x))),Pval=anova(x)[2,4])})

In [ ]:
univ_results

In [ ]:
# drop none
# whole case only
data <- na.omit(data)
length(data$CAN_ABO)

In [ ]:
# Split in validation and training data, seed for reproducability, choosen randomly

## 80% of the sample size
smp_size <- floor(0.80 * nrow(data))

## set the seed to make your partition reproducible
set.seed(123)
train_ind <- sample(seq_len(nrow(data)), size = smp_size)

train <- data[train_ind, ]
test2 <- data[-train_ind, ]

data <- train

In [ ]:
str(data)

In [ ]:
plot(survfit(Surv(time, status) ~ 1, data = data), 
     xlab = "Days", 
     ylab = "Graft survival probability")

In [ ]:
res.cox <- coxph(Surv(time, status) ~ CAN_GENDER+REC_DGN+CAN_ABO+CAN_DIAB_TY+CAN_MALIG+DON_ABO+DON_AGE+
                 REC_AGE_AT_TX+REC_MM_EQUIV_CUR+REC_MM_EQUIV_TX+REC_COLD_ISCH_TM+REC_CREAT+
                 REC_DISCHRG_CREAT+decade, data = data)

....Forum answer: "When I asked Terry Therneau (author of pkg:survival) about that several years ago he said the test that is being triggered to generate that warning is overly sensitive. Generally the warning is not correct. You can usually just look at your coefficients to see that they are not infinite or even effectively infinite which a coefficient of 20 would be unrealistically large in most instances"....
=> Concerning the error message


In [ ]:
summary(res.cox)

In [ ]:
ggsurvplot(survfit(res.cox),data=data, palette = "#2E9FDF",
           ggtheme = theme_minimal(), ylab = c("Propability of graft survival"))

In [ ]:
concordance(res.cox, newdata = test2)
concordance(res.cox)
extractAIC(res.cox)
print("AIC:")
extractAIC(res.cox)

# parametric part

In [ ]:
data$time[data$time==0]<-0.00001

In [ ]:
s <- Surv(time, status) ~ CAN_GENDER+REC_DGN+CAN_ABO+CAN_DIAB_TY+CAN_MALIG+DON_ABO+DON_AGE+
                 REC_AGE_AT_TX+REC_MM_EQUIV_CUR+REC_MM_EQUIV_TX+REC_COLD_ISCH_TM+REC_CREAT+
                 REC_DISCHRG_CREAT+decade

In [ ]:
reg_fit <- coxph(Surv(time, status)~1, data=data)
summary(reg_fit)

# Now get baseline curve
baseline <- basehaz(reg_fit)

In [ ]:
plot(baseline$time, baseline$hazard)
head(baseline)

In [ ]:
fitwb <- survreg(s, data=data, dist = "weibull") 
fitexp <- survreg(s, data = data, dist = "exponential")

fitgaus <- survreg(s, data = data, dist = "gaussian")
fitlog <- survreg(s, data = data, dist = "logistic")

fitlognorm <- survreg(s, data = data, dist = "lognormal")
fitloglog <- survreg(s, data = data, dist = "loglogistic")

In [ ]:
a<-extractAIC(fitwb)
b<-extractAIC(fitexp)
c<-extractAIC(fitgaus)
d<-extractAIC(fitlog)
e<-extractAIC(fitlognorm)
f<-extractAIC(fitloglog)
g <- extractAIC(res.cox)
aicput <- rbind(a,b,c,d,e,f,g)
namemod <- c("Weibull", "Exponential", "Gaussian", "Logistic", "Lognormal", "Loglogistic", "Cox")
aicput <- cbind(aicput, namemod)
aicput


In [ ]:
concordance(fitwb)
concordance(fitwb, newdata = test2)

In [ ]:
concordance(fitloglog)
concordance(fitloglog, newdata =test2)
# summary(fitloglog)

In [ ]:
#anova(fitwb)
summary(fitwb)

In [ ]:
s <- with(data,Surv(time,status))
sWei <- survreg(s ~ as.factor(CAN_GENDER),dist='weibull',data=data)


In [ ]:
sWei

In [ ]:
intercept<-10.1719889
scale<-1.15

par(mfrow=c(1,2),mar=c(5.1,5.1,4.1,2.1)) # Make room for the hat.
# S(t), the survival function
curve(pweibull(x, scale=exp(intercept), shape=1/scale, lower.tail=FALSE), 
      from=0, to=4000, col='red', lwd=2, ylab=expression(hat(S)(t)), xlab='t',bty='n',ylim=c(0,1))
# h(t), the hazard function
curve(dweibull(x, scale=exp(intercept), shape=1/scale)
      /pweibull(x, scale=exp(intercept), shape=1/scale, lower.tail=FALSE), 
      from=0, to=4000, col='blue', lwd=2, ylab=expression(hat(h)(t)), xlab='t',bty='n')
par(mfrow=c(1,1),mar=c(5.1,4.1,4.1,2.1))

In [ ]:
wbmod <- fitwb
s <- seq(.01, .99, by = .01)
t_0 <- predict(wbmod, newdata = data.frame(x = 0), 
                 type = "quantile", p = s)
t_1 <- predict(wbmod, newdata = data.frame(x = 1), 
                type = "quantile", p = s)

smod <- data.frame(time = c(t_0, t_1), 
                   surv = rep(1 - s, times = 2), 
                   strata = rep(c(0, 1), each = length(s)),
                   upper = NA, lower = NA)

head(surv_summary(cm))

library("ggplot2")
ggplot() + 
  geom_line(data = smod, aes(x = time, y = surv, color = factor(strata))) +
  theme_classic()